In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv("C:\\Users\\usman\\Documents\\train.csv")
df.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [3]:
train=df.drop(columns=["SalePrice"])

In [4]:
x_train=df.drop(columns=["SalePrice"])
y_train=df["SalePrice"]

In [5]:
num_vars=x_train.select_dtypes(include=['float64',"int64"]).columns
num_vars
isnull=x_train.select_dtypes(include=['float64',"int64"]).isnull().sum()
isnull

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [56]:
#[var for var in x_train[num_vars] if x_train[var].isnull().sum()>0]

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [6]:
missing_data_num_vars=[var for var in num_vars if isnull[var].sum()>0 ]
missing_data_num_vars

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [58]:
#for categorical data
cat_vars=x_train.select_dtypes(include=["O"]).columns
cat_vars

null=x_train.select_dtypes(include=["O"]).isnull().sum()


missing_data_cat_vars=[var for var in cat_vars if null[var].sum()>0 ]
missing_data_cat_vars

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [8]:
#creating pipeline for each attribute
num_mean=['LotFrontage']
num_median=['MasVnrArea', 'GarageYrBlt']
cat_mode=['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish']
cat_missing=['GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [9]:
num_mean_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="mean"))])
num_median_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])
                      

In [10]:
cat_mode_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])
cat_missing_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="constant",fill_value="missing"))])   

In [11]:
#to implement this above created pipeline we use column_transformer

In [12]:
#transforming categorical columns
ColumnTransformer(transformers=[("mode_imputer"),cat_mode_imputer,cat_mode])
ColumnTransformer(transformers=[("missing_imputer"),cat_missing_imputer,cat_missing])

ColumnTransformer(transformers=['missing_imputer',
                                Pipeline(steps=[('imputer',
                                                 SimpleImputer(fill_value='missing',
                                                               strategy='constant'))]),
                                ['GarageQual', 'GarageCond', 'PoolQC', 'Fence',
                                 'MiscFeature']])

In [13]:
#transforming numeric columns
ColumnTransformer(transformers=["mean_imputer",num_mean_imputer,num_mean])
ColumnTransformer(transformers=["median_imputer",num_median_imputer,num_median])

ColumnTransformer(transformers=['median_imputer',
                                Pipeline(steps=[('imputer',
                                                 SimpleImputer(strategy='median'))]),
                                ['MasVnrArea', 'GarageYrBlt']])

In [14]:
#transforming all categorical and numeric data and assigning to a variable

In [15]:
pre_process=ColumnTransformer(transformers=[("mean_imputer",num_mean_imputer,num_mean),
                               ("median_imputer",num_median_imputer,num_median),
                               ("missing_imputer",cat_missing_imputer,cat_missing),
                               ("mode_imputer",cat_mode_imputer,cat_mode)
                               
                               ])


In [16]:
pre_process

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('missing_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant'))]),
                                 ['GarageQual', 'GarageCond', 'PoolQC', 'Fence',
                                  'MiscFeature']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                         

In [17]:
#now fitting on data
pre_process.fit(x_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('missing_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant'))]),
                                 ['GarageQual', 'GarageCond', 'PoolQC', 'Fence',
                                  'MiscFeature']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                         

In [18]:
x_train_clean=pre_process.transform(x_train)

In [19]:
#now we want to see in which column(variable) what we have done so far
pre_process.named_transformers_["mean_imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [20]:
x_train_clean

array([[65.0, 196.0, 2003.0, ..., 'Gd', 'Attchd', 'RFn'],
       [80.0, 0.0, 1976.0, ..., 'TA', 'Attchd', 'RFn'],
       [68.0, 162.0, 2001.0, ..., 'TA', 'Attchd', 'RFn'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'Gd', 'Attchd', 'RFn'],
       [68.0, 0.0, 1950.0, ..., 'Gd', 'Attchd', 'Unf'],
       [75.0, 0.0, 1965.0, ..., 'Gd', 'Attchd', 'Fin']], dtype=object)

In [23]:
#it droped those columns in x_train data set in which there is no nan value was present so we want back all those columns too
pre_process.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('missing_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu',
   'GarageType',
   'GarageFinish']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
 

In [26]:
missing_df=pd.DataFrame(x_train_clean,columns=num_mean+num_median+cat_mode+cat_missing)
missing_df.shape

(1460, 19)

In [35]:
Non_missing_df=x_train.drop(columns=missing_df.columns)
Non_missing_df

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,1457,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,1458,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,1459,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [30]:
missing_df.shape

(1460, 61)

In [45]:
x_train_df=pd.concat([Non_missing_df,missing_df],axis="columns")

In [46]:
x_train_df.shape

(1460, 80)